# Imports
Import all needed libraries

In [ ]:
import os
import librosa
import matplotlib.pyplot as plt
import random

# Function to Generatepectogram
Function to generate the spectrogram.


In [ ]:
def generate_spectrogram(y, sr, n_fft, hop_length, window):
    D = librosa.stft(y, n_fft=n_fft, hop_length=hop_length, window=window)
    S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)

    plt.figure(figsize=(12, 8))
    librosa.display.specshow(S_db, sr=sr, hop_length=hop_length, x_axis='time', y_axis='log', cmap='viridis')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Spectrogram')
    #plt.show()
    return plt


# Selecting Samples
selects randomg samples from the song, and sets the naming conventions for the outputted spectrograms.


In [ ]:

# Define the source and output directories
source_dir = "/content/music_files"  # Change this to your directory in Colab
output_dir = "/content/audio_spects_2"  # Directory to save spectrograms

# Ensure the output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Define STFT parameters
n_fft = 1024  # Number of FFT points, affects frequency resolution
hop_length = 256  # Hop length, affects time resolution
window = 'hann'  # Window type

for file_name in os.listdir(source_dir):
    if file_name.lower().endswith(('.wav', '.mp3', '.flac', '.ogg')):
        spectrogram_name = f"{os.path.splitext(file_name)[0]}.png"
        output_path = os.path.join(output_dir, spectrogram_name)

        # Check if the spectrogram already exists
        if os.path.exists(output_path):
            print(f"Spectrogram already exists at {output_path}. Skipping...")
            continue

        file_path = os.path.join(source_dir, file_name)  # Full path to the audio file

        try:
            # Load the entire audio file
            y, sr = librosa.load(file_path, sr=None)

            # Ensure the audio is at least 30 seconds long
            if len(y) / sr >= 30:
                # Randomly select a 30-second segment
                total_duration = len(y) / sr
                start_time = random.uniform(0, total_duration - 30)
                end_time = start_time + 30

                # Slice the segment from the full audio
                start_sample = int(start_time * sr)
                end_sample = int(end_time * sr)
                y_segment = y[start_sample:end_sample]

                # Generate the spectrogram for the segment
                plt = generate_spectrogram(y_segment, sr, n_fft, hop_length, window)

                # Save the spectrogram
                plt.savefig(output_path)
                plt.close()

                print(f"Spectrogram saved at {output_path}")

        except Exception as e:
            print(f"Failed to process {file_name}: {e}")